In [ ]:
import pandas as pd
import numpy as np
import math
import random

In [ ]:
path_sysprecau = "https://drive.google.com/file/d/1Axrc1KICO1f7VG0Rknrja35rOkAfQdFH/view?usp=sharing"
path_dataset = '/content/drive/MyDrive/Compfest Dataset/dataset.csv'

Data = pd.read_csv(path_dataset)
Data = Data.fillna('None')
data = Data[:15]

In [ ]:
data.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,Fungal infection,itching,skin_rash,dischromic _patches,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [ ]:
len(Data)

4920

'''
1. iterate every row
2. check how symptons for particular row
    - if symptons number <0 N <= 4 , generate 1 list random with 4 symptons
    - if symptons number <4 N <= 8 , generate 2 list random with 4 symptons
    - if symptons number <8 N <= 16 , generate 3 list random with 4 symptons
    - if symptons number <16 N , generate 4 list random with 4 symptons
4. all generated list append to new column called `symptons_list`
'''


In [ ]:
def get_random_symptons(input_list,number):
    # Shuffle the input list to randomize its order
    randomized_list = random.sample(input_list, len(input_list))

    # Take the first four elements from the randomized list
    random_symptons = randomized_list[:number]
    return random_symptons

In [ ]:
def preprocess(data):
  new_data = []
  for i,row in data.iterrows():
    label = row.loc['Disease']
    count_none = row.isin(['None']).sum()
    count_sympton = 17 - count_none

    list_of_sympton = list(row.iloc[1:(count_sympton+1)])

    # delete underscore
    list_of_sympton = [word.replace('_', ' ') for word in list_of_sympton]

    # print(f'{i}: {label} = {count_sympton} --> {list_of_sympton}')

    number = random.choice([4, 5, 6,7])
    list_to_add = get_random_symptons(list_of_sympton,number=number)
    list_to_add = ', '.join(list_to_add)

    if count_sympton > 0 and count_sympton <= 4:
      new_data.append([label, list_to_add])
    elif count_sympton > 4 and count_sympton <= 8:
      for j in range (2):
        new_data.append([label, list_to_add])
    elif count_sympton > 8 and count_sympton <= 15:
      for j in range (4):
        new_data.append([label, list_to_add])
    elif count_sympton > 15:
      for j in range (5):
        new_data.append([label, list_to_add])

  return new_data



In [ ]:
columns = ['diseases', 'symptons']
new_data = preprocess(Data)

df = pd.DataFrame(new_data, columns=columns)
len(df)

12210

In [ ]:
df.to_csv('output_file.csv', index=False)

## Clinical bert embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# from sentence-transformers
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

sentences = ['ratu',
             'raja']

encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)

sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


In [ ]:

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    similarity = dot_product / (norm_A * norm_B)
    return similarity

similarity_score = cosine_similarity(sentence_embeddings[0], sentence_embeddings[1])
print("Cosine Similarity:", np.round(similarity_score,2))

Cosine Similarity: 0.91


In [ ]:
sentence_embeddings

tensor([[ 0.4744,  0.1135,  0.1444,  ...,  0.1275,  0.2114, -0.0558],
        [ 0.7707,  0.0464, -0.1112,  ..., -0.0583,  0.5149, -0.0863]])